In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import MatchingAlgorithm 

import importlib 
importlib.reload(MatchingAlgorithm)


<module 'MatchingAlgorithm' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\MatchingAlgorithm.py'>

In [2]:
import FeatureEngineering
importlib.reload(FeatureEngineering)


<module 'FeatureEngineering' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\FeatureEngineering.py'>

### Running algorithm

In [3]:
# INPUT YOUR API KEY

API_key = ''

In [4]:
orders_df = pd.read_csv("Simulated Data/order_data_1576.csv")
catchments_df = pd.read_csv("Simulated Data/catchment_data_1576.csv")
phlebs_df = pd.read_csv("Simulated Data/phleb_data_1576.csv")

In [5]:
#orders_df = orders_df.iloc[:15]
#phlebs_df = phlebs_df.iloc[:2]
catchments_df = catchments_df.iloc[:1] #remove this for multi-ends 

In [69]:
coordinate_list = FeatureEngineering.get_coordinates_list(orders_df, catchments_df, phlebs_df)
time_matrix =  FeatureEngineering.create_time_matrix(coordinate_list, API_key)

In [6]:
time_matrix = np.loadtxt('timeMatrix.txt')

In [129]:
output = MatchingAlgorithm.run_algorithm_version_timeMatrix(orders_df, catchments_df, phlebs_df, time_matrix)
print(output)

{
  "Metadata": {
    "Locations": [
      {
        "Location Index": 0,
        "Coordinate": "28.453012121886808,77.04885293025431",
        "Order Id": "Ending Location"
      },
      {
        "Location Index": 1,
        "Coordinate": "28.446275918004297,77.05666911676877",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 2,
        "Coordinate": "28.436374190228392,77.10818368839077",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 3,
        "Coordinate": "28.46043105269203,77.12523221919234",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 4,
        "Coordinate": "28.44673371496172,77.08386447523183",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 5,
        "Coordinate": "28.450267421182843,77.0411748655223",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 6,
        "Coordinate": "28.448895764608586,7

### Prescriptive Analytics

In [7]:
phlebs_df['service_rating'] = 5 #ensure all service rating is the same

In [8]:
demand_ratios = [(1, 0, 0), (0.8, 0.2, 0), (0.6, 0.4, 0),  (0.6, 0.2, 0.2), (0.4, 0.4, 0.2), (0.2, 0.6, 0.2), (0, 0.6, 0.4), (0, 0.8, 0.2), (0, 1, 0)
                 , (0, 0.8, 0.2), (0, 0.6, 0.4), (0, 0.4, 0.6), (0, 0.2, 0.8), (0, 0, 1)]

full_ratios = []

for demand in demand_ratios:
    list_of_ratios, list_of_total_transit_time, list_of_total_cost, list_of_numbers, list_of_jsons, list_of_chosen_phlebs = MatchingAlgorithm.run_prescriptive_analysis(
        orders_df, catchments_df, phlebs_df, time_matrix, demand[0], demand[1], demand[2])
    full_ratios.append(list_of_ratios)



In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

mean_ratios = [np.mean(list_of_ratios, axis=0) for list_of_ratios in full_ratios]

fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[0] for ratio in mean_ratios]
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[1] for ratio in mean_ratios]
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[2] for ratio in mean_ratios]
), row=3, col=1)


fig.update_layout(height=800, width=1300, title_text="Demand & Supply Ratios")
fig.show()

### Visualisation

In [7]:
# import pygeos
import RouteVisualisation
import geopandas as gpd

importlib.reload(RouteVisualisation)

gpd.options.use_pygeos = False

In [8]:
addresses_list = FeatureEngineering.get_coordinates_list(orders_df, catchments_df, phlebs_df)

In [9]:
json_result = json.loads(output)

polygon = gpd.read_file("Simulation\Gurugram_sample_Polygon.geojson")
addresses_list = addresses_list

RouteVisualisation.visualise_routes(json_result, polygon)

Creating route travelled by Phlebotomist ID #0
[1, 126, 36, 127, 0]
Coords: 28.445671728377317,77.05941330924517
Node:  6194749658 4140363615
Route:  [6194749658, 515774489, 317145222, 515774494, 312919889, 3503757322, 9903465387, 312919890, 2008318332, 6485206039, 2008318465, 4223749839, 582869715, 582870142, 4223749840, 7513729476, 2008318533, 6191422035, 266381230, 9918424896, 7513729472, 7513729501, 4140363611, 4140363616, 4140363595, 4140363615]
Drawing Phlebotomist ID #0's route...
Map created
Coords: 28.448023577545065,77.04923340328399
Node:  4140363615 4140363683
Route:  [4140363615, 4140363595, 4140363616, 4140363611, 7513729501, 7513729472, 9918424896, 266381230, 4140363667, 4140363683]
Coords: 28.449817802401363,77.05020131498193
Node:  4140363683 4233033187
Route:  [4140363683, 4140363696, 4206682721, 4140363706, 4140363708, 4140363716, 268158707, 4233038242, 4233033187]
Coords: 28.450761009875706,77.04672322432197
Node:  4233033187 7456856376
Route:  [4233033187, 42330382